In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [9]:
#import the world_happiness csv
world_happiness = "data/world_happiness.csv"
world_happiness_df = pd.read_csv(world_happiness)
world_happiness_df.head()

,Year,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Dystopia.Residual
0,2019,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,4.054
1,2019,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,3.777
2,2019,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,3.729
3,2019,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,3.781
4,2019,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,3.790


In [10]:
#import the annual_working_hours_per_worker
annual_working_hours = "data/annual_working_hours_per_worker.csv"
annual_work_df = pd.read_csv(annual_working_hours)
annual_work_df.head()

,Entity,Code,Year,Average annual working hours per worker
0,Argentina,ARG,1950,2034.0000
1,Argentina,ARG,1951,2037.8667
2,Argentina,ARG,1952,2041.7408
3,Argentina,ARG,1953,2045.6223
4,Argentina,ARG,1954,2049.5112


### Transform world_happiness DataFrame

### Transform annual_work DataFrame

### Create database connection

### Load DataFrames into database